In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_vccsddl52',
    proc_date,
    'temp_sap_vccsddl52'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_sap_bw.sap_vccsddl52(
    hash_id BIGINT,
    reqtsn STRING,
    datapakid STRING,
    record BIGINT,
    calday DATE,
    do_number STRING,
    do_item STRING,
    creation_date DATE,
    gi_date DATE,
    act_gi_dte DATE,
    deleted_record STRING,
    product_id STRING,
    deliver_qty_sale DECIMAL(38,9),
    sales_unit STRING,
    deliver_qty_base DECIMAL(38,9),
    base_unit STRING,
    dc_id STRING,
    sto_number STRING,
    sto_item STRING,
    sts_pick STRING,
    osts_pick STRING,
    tgoodmv_st STRING,
    pick_conf STRING,
    goodsmv_st STRING,
    recordmode STRING,
    unload_pt STRING,
    imounl_pt STRING,
    sales_dist STRING,
    bill_block STRING,
    incoterms STRING,
    incoterms2 STRING,
    imoincotm2 STRING,
    cust_group STRING,
    del_type STRING,
    ship_date STRING,
    creditor STRING,
    del_block STRING,
    load_pt STRING,
    route STRING,
    doc_categ STRING,
    ch_on STRING,
    rt_promo STRING,
    batch STRING,
    eanupc STRING,
    createdby STRING,
    crea_time STRING,
    bilblk_dl STRING,
    bus_area STRING,
    pick_indc STRING,
    cust_grp1 STRING,
    cust_grp2 STRING,
    cust_grp3 STRING,
    cust_grp4 STRING,
    cust_grp5 STRING,
    consu_flag STRING,
    whse_num STRING,
    stor_loc STRING,
    plant STRING,
    strge_bin STRING,
    strge_type STRING,
    itm_type STRING,
    forwagent STRING,
    item_categ STRING,
    per_resp STRING,
    stat_date STRING,
    prvdoc_ctg STRING,
    imodoccat STRING,
    wbs_elemt STRING,
    del_wa_dh STRING,
    calmonth STRING,
    calquarter STRING,
    calyear STRING,
    fiscvarnt STRING,
    fiscper STRING,
    fiscyear STRING,
    upd_date STRING,
    no_del_it DECIMAL(38,9),
    denomintr DECIMAL(38,9),
    numerator DECIMAL(38,9),
    shp_pr_tmf DECIMAL(38,9),
    shp_pr_tmv DECIMAL(38,9),
    proc_date DATE,
    file_creation_ts TIMESTAMP
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
) 
""")

In [0]:
spark.sql(f"""
    MERGE INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_vccsddl52 main_tbl USING (
    WITH main AS (
        SELECT
            reqtsn,
            datapakid,
            record,
            TO_DATE(calday, 'yyyyMMdd') AS calday,
            deliv_numb AS do_number,
            deliv_item AS do_item,
            TO_DATE(createdon, 'yyyyMMdd') AS creation_date,
            TO_DATE(gi_date, 'yyyyMMdd') AS gi_date,
            try_cast(act_gi_dte as DATE) AS act_gi_dte,
            IF(recordmode = 'R', 'Y', NULL) deleted_record,
            LTRIM('0', material) product_id,    
            dlv_qty deliver_qty_sale,
            sales_unit,
            act_dl_qty deliver_qty_base,
            base_uom base_unit,
            ship_point dc_id,
            refer_doc sto_number,
            refer_itm sto_item,
            IF(sts_pick = '', NULL, sts_pick) sts_pick,
            IF(osts_pick = '', NULL, osts_pick) osts_pick,
            IF(tgoodmv_st = '', NULL, tgoodmv_st) tgoodmv_st,
            IF(pick_conf = '', NULL, pick_conf) pick_conf,
            IF(goodsmv_st = '', NULL, goodsmv_st) goodsmv_st,
            recordmode,
            unload_pt,
            imounl_pt,
            sales_dist,
            bill_block,
            incoterms,
            incoterms2,
            imoincotm2,
            cust_group,
            del_type,
            ship_date,
            creditor,
            del_block,
            load_pt,
            route,
            doc_categ,
            ch_on,
            rt_promo,
            batch,
            eanupc,
            createdby,
            crea_time,
            bilblk_dl,
            bus_area,
            pick_indc,
            cust_grp1,
            cust_grp2,
            cust_grp3,
            cust_grp4,
            cust_grp5,
            consu_flag,
            whse_num,
            stor_loc,
            plant,
            strge_bin,
            strge_type,
            itm_type,
            forwagent,
            item_categ,
            per_resp,
            stat_date,
            prvdoc_ctg,
            imodoccat,
            wbs_elemt,
            del_wa_dh,
            calmonth,
            calquarter,
            calyear,
            fiscvarnt,
            fiscper,
            fiscyear,
            upd_date,
            no_del_it,
            denomintr,
            numerator,
            shp_pr_tmf,
            shp_pr_tmv,
            proc_date,
            file_creation_ts
        FROM temp_sap_vccsddl52
        QUALIFY ROW_NUMBER() OVER (PARTITION BY reqtsn, datapakid, record ORDER BY proc_date DESC) = 1 
    )
    SELECT 
        farm_fingerprint(CONCAT(
            IFNULL(CAST(calday AS STRING), ""),
            IFNULL(CAST(creation_date AS STRING), ""),
            IFNULL(CAST(gi_date AS STRING), ""),
            IFNULL(CAST(act_gi_dte AS STRING), ""),
            IFNULL(CAST(do_number AS STRING), ""),
            IFNULL(CAST(do_item AS STRING), ""),
            IFNULL(CAST(product_id AS STRING), ""),
            IFNULL(CAST(sales_unit AS STRING), ""),
            IFNULL(CAST(base_unit AS STRING), ""),
            IFNULL(CASE WHEN CAST(deliver_qty_sale AS DOUBLE) = CAST(CAST(deliver_qty_sale AS DOUBLE) AS BIGINT) THEN CAST(CAST(deliver_qty_sale AS BIGINT) AS STRING) ELSE TRIM(TRAILING '0' FROM(CAST(deliver_qty_sale AS DECIMAL(38,10)))) END, ""),
            IFNULL(CASE WHEN CAST(deliver_qty_base AS DOUBLE) = CAST(CAST(deliver_qty_base AS DOUBLE) AS BIGINT) THEN CAST(CAST(deliver_qty_base AS BIGINT) AS STRING) ELSE TRIM(TRAILING '0' FROM(CAST(deliver_qty_base AS DECIMAL(38,10)))) END, ""),
            IFNULL(CAST(dc_id AS STRING), ""),
            IFNULL(CAST(sto_number AS STRING), ""),
            IFNULL(CAST(sto_item AS STRING), ""),
            IFNULL(CAST(sts_pick AS STRING), ""),
            IFNULL(CAST(osts_pick AS STRING), ""),
            IFNULL(CAST(goodsmv_st AS STRING), ""),
            IFNULL(CAST(tgoodmv_st AS STRING), ""),
            IFNULL(CAST(deleted_record AS STRING), "")
        )) hash_id,
        *
    FROM main
) temp_tbl
ON main_tbl.hash_id = temp_tbl.hash_id
WHEN NOT MATCHED THEN INSERT *
""")